# Figure 3 — Noise-Shaped Synchrony in a FitzHugh–Nagumo Network

This notebook reproduces **Figure 3** of the manuscript  
*"Noise-shaped Synchrony in Neuronal Oscillator Networks"*.

It demonstrates how **noise shaping influences collective synchrony** in a network of coupled FitzHugh–Nagumo (FHN) oscillators using precomputed simulation data.

---

## What this notebook does

- Loads precomputed network simulation results from CSV files  
- Computes and visualizes synchrony-related network measures  
- Generates the final multi-panel figure shown in the manuscript

This notebook **does not re-run large-scale network simulations** and is therefore safe to execute on standard hardware.

---

## Input data

The following data files (located in this folder) are used as inputs:

- Network simulation output CSV files stored in this directory

---

## Output

Running all cells will generate the figure files corresponding to **Figure 3** in the manuscript.

---

## How to run

Simply execute all cells from top to bottom:

- Jupyter menu: **Kernel → Restart & Run All**
- Or execute cells sequentially

Typical runtime on a standard laptop: **seconds to a few minutes**, depending on the size of the loaded datasets.


In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import matplotlib as mpl

mpl.rcParams.update({'text.usetex': True, 'font.family': 'serif'})
USE_TEX = True
FIGSIZE = (12.5,7.5)
DPI = 300
MAIN_CSV = "sc_c_00600_clusters_.csv"
ACF1 = "acf_average_n_00000_c_00600_.csv"
ACF2 = "acf_average_n_00200_c_00600_.csv"
ACF3 = "acf_average_n_04000_c_00600_.csv"
ACF4 = "acf_average_n_00025_c_00600_.csv"
ACF1_RECT = [0.20,0.54,0.60,0.12]
ACF2_RECT = [0.20,0.39,0.60,0.12]
ACF3_RECT = [0.20,0.24,0.60,0.12]
ACF4_RECT = [0.20,0.09,0.60,0.12]
W_SCALE = 0.5
for R in (ACF1_RECT,ACF2_RECT,ACF3_RECT,ACF4_RECT):
    R[2]*=W_SCALE
MAIN_AX_RECT = [0.08,0.10,0.84,0.85]
PALETTE = {0:"#e41a1c",1:"#377eb8",2:"#4daf4a",3:"#984ea3",4:"#ff7f00"}
DEFAULT_LABEL_OFFSETS = {"(a)":(-30,15),"(b)":(20,15),"(c)":(20,15),"(d)":(35,15)}
INSET_N = {"(a)":1e-4,"(b)":2.5e-3,"(c)":2.0e-2,"(d)":4.0e-1}
ANCHOR_D = (14.5e-5,0.5)
ANCHOR_B = (1.5e-2,1.3)
ANCHOR_C = (100,0.7)
ANCHOR_A = (20e-5,1.3)
def _nice_step(n,target_ticks=6):
    if n<=0:return 1
    raw=max(1,n/target_ticks)
    exp=int(np.floor(np.log10(raw))) if raw>0 else 0
    frac=raw/(10**exp)
    if frac<=1:nice=1
    elif frac<=2:nice=2
    elif frac<=5:nice=5
    else:nice=10
    return max(1,int(nice*(10**exp)))
def load_acf_series(path):
    return np.loadtxt(path,dtype=float,ndmin=1)
def draw_acf_panel(fig,rect,arr,period,tau,label,color,linestyle):
    ax=fig.add_axes(rect,zorder=5)
    lags=np.arange(len(arr))
    cycles=lags/period
    available=(len(arr)-1)/period if len(arr)>1 else 0.0
    xmax=available if tau is None else min(tau,available)
    ax.plot(cycles,arr,linewidth=2.0,alpha=0.95,color=color,linestyle="-",label=label)
    ax.set_xlim(0.0,xmax)
    ax.set_ylim(-0.5,1.50)
    ax.set_yticks([-0.5,0.0,1.0])
    ax.grid(True,which="major",linestyle=":",linewidth=0.8,alpha=0.6)
    ax.tick_params(axis="both",labelsize=20,direction="in")
    whole=int(np.floor(xmax))
    step=_nice_step(whole,6)
    ax.set_xticks(np.arange(0,whole+1,step))
    ax.set_xlabel(r"$\tau/T$",fontsize=24)
    ax.set_ylabel(r"$\rho$",fontsize=24,rotation=0,labelpad=20)
    return ax,xmax
def _nearest_index_logaware(ax,x_array,target):
    x=np.asarray(x_array,dtype=float)
    if x.size==0 or not np.all(np.isfinite(x)):return 0
    xmin,xmax=np.nanmin(x),np.nanmax(x)
    if target<=xmin:return int(np.nanargmin(x))
    if target>=xmax:return int(np.nanargmax(x))
    use_log=(ax.get_xscale()=="log") and (target>0) and np.all(x>0)
    dist=np.abs(np.log(x)-np.log(target)) if use_log else np.abs(x-target)
    return int(np.nanargmin(dist))
def add_inset_arrows(ax,x_noise,y_vals,inset_n,arrow_alpha=0.6,arrow_lw=4.0,arrow_ms=28,font_size=13):
    x_noise=np.asarray(x_noise,dtype=float)
    y_vals=np.asarray(y_vals,dtype=float)
    for lab,n_val in inset_n.items():
        idx=_nearest_index_logaware(ax,x_noise,float(n_val))
        xy=(x_noise[idx],y_vals[idx])
        offset=DEFAULT_LABEL_OFFSETS.get(lab,(8,8))
        ax.annotate(lab,xy=xy,xycoords="data",xytext=offset,textcoords="offset points",
                    ha="left",va="bottom",fontsize=font_size,color="black",alpha=0.9,
                    arrowprops=dict(arrowstyle="->",color="black",alpha=arrow_alpha,
                                    lw=arrow_lw,mutation_scale=arrow_ms),zorder=6,annotation_clip=False)
df=pd.read_csv(MAIN_CSV)
y_candidates=["mfr","spike_contrast","sc","psi"]
ycol=None
for k in y_candidates:
    if k in df.columns:ycol=k;break
D=df["noise"].to_numpy()
Y=df[ycol].to_numpy()
clusters=df["cluster"].astype(int).to_numpy()
point_colors=[PALETTE.get(int(k),"#808080") for k in clusters]
fig=plt.figure(figsize=FIGSIZE)
ax=fig.add_axes(MAIN_AX_RECT)
ax.plot(D,Y,linestyle="--",color="black",linewidth=1.5)
ax.scatter(D,Y,s=100,c=point_colors,edgecolors="black",linewidths=0.5)
ax.set_xscale("log")
ax.set_xticks([1e-4,1e-3,1e-2,1e-1])
ax.set_xticklabels([r"$10^{-4}$",r"$10^{-3}$",r"$10^{-2}$",r"$10^{-1}$"],fontsize=22)
ax.set_xlabel(r"$D$",fontsize=24)
ax.set_xlim(8e-5,7e-1)
ax.set_ylabel(r"$\kappa$",fontsize=24,rotation=0,labelpad=20)
ax.set_ylim(0,1.6)
ax.grid(True,which="major",linestyle=":",linewidth=0.8,alpha=0.6)
ax.tick_params(axis="both",labelsize=22,direction="in")
add_inset_arrows(ax,D,Y,INSET_N,0.6,2.0,20,20)
xA_fig,yA_fig=fig.transFigure.inverted().transform(ax.transData.transform(ANCHOR_A))
xB_fig,yB_fig=fig.transFigure.inverted().transform(ax.transData.transform(ANCHOR_B))
xC_fig,yC_fig=fig.transFigure.inverted().transform(ax.transData.transform(ANCHOR_C))
xD_fig,yD_fig=fig.transFigure.inverted().transform(ax.transData.transform(ANCHOR_D))
ACF1_RECT[0]=max(0.0,min(1.0-ACF1_RECT[2],xA_fig))
ACF1_RECT[1]=max(0.0,min(1.0-ACF1_RECT[3],yA_fig))
ACF2_RECT[0]=max(0.0,min(1.0-ACF2_RECT[2],xB_fig))
ACF2_RECT[1]=max(0.0,min(1.0-ACF2_RECT[3],yB_fig))
ACF3_RECT[0]=max(0.0,min(1.0-ACF3_RECT[2],xC_fig))
ACF3_RECT[1]=max(0.0,min(1.0-ACF3_RECT[3],yC_fig))
ACF4_RECT[0]=max(0.0,min(1.0-ACF4_RECT[2],xD_fig))
ACF4_RECT[1]=max(0.0,min(1.0-ACF4_RECT[3],yD_fig))
ACF3_RECT[2]*=0.7
for R in (ACF1_RECT,ACF2_RECT,ACF3_RECT,ACF4_RECT):
    R[2]*=0.9
periods=[431.0,431.0,431.0,431.0]
taus=[10,30,3,45]
labels=["CS","NS","PN","NS"]
acf_files=[Path(ACF1),Path(ACF2),Path(ACF3),Path(ACF4)]
rects=[ACF1_RECT,ACF2_RECT,ACF3_RECT,ACF4_RECT]
styles=[("#984ea3","-."),("#4daf4a",":"),("#e41a1c","--"),("#4daf4a","-")]
panel_letters=["(a)","(c)","(d)","(b)"]
for i,(fpath,period,tau,label,rect,(color,ls)) in enumerate(zip(acf_files,periods,taus,labels,rects,styles),start=1):
    if not fpath.exists():
        axp=fig.add_axes(rect,zorder=5);axp.text(0.5,0.5,f"Missing:\n{fpath.name}",ha="center",va="center",fontsize=15,transform=axp.transAxes);axp.set_axis_off();continue
    arr=load_acf_series(fpath)
    axp,_=draw_acf_panel(fig,rect,arr,period,tau,label,color,ls)
    axp.text(0.01,0.94,panel_letters[i-1],transform=axp.transAxes,ha="left",va="top",fontsize=18)
out=Path("fig_03.png")
fig.savefig(out,dpi=DPI,bbox_inches="tight")
plt.show()
print(out.resolve().as_posix())
